# Counterparty Credit Risk & Backtesting MVP

**Head of CCR Level Demo** – Pure Python | Zero Paid Services | Runs anywhere

As-of date: **25 Nov 2025** | 2,500 Monte Carlo paths | 95% PFE

In [ ]:
import pandas as pd
from core.exposure import CCRSimulator
import matplotlib.pyplot as plt
%matplotlib inline

trades = pd.read_csv('../data/sample_portfolio.csv', parse_dates=['maturity_date'])
sim = CCRSimulator(n_paths=2500)
profiles = sim.run_netting_set(trades)

print(f"Successfully simulated {len(profiles)} netting sets")
profiles['NS001'].head()

## Exposure Profile – NS001 (Deutsche Bank – Investment Grade)

In [ ]:
df = profiles['NS001']
plt.figure(figsize=(12,6))
plt.plot(df['time'], df['EE_total']/1e6, label='Expected Exposure (EE)', linewidth=3, color='#2E8B57')
plt.plot(df['time'], df['PFE_total']/1e6, label='PFE 95%', linewidth=3, color='#DC143C')
plt.title('Netting Set NS001 – Deutsche Bank (FX Forward + IRS)', fontsize=16)
plt.xlabel('Time Horizon (years)', fontsize=12)
plt.ylabel('Exposure (€ millions)', fontsize=12)
plt.legend(fontsize=12)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## Quick Limit Check (Single-Name PFE Limit = €1.2m)

In [ ]:
limit = 1_200_000
peak_pfe = profiles['NS001']['PFE_total'].max()
status = "BREACH" if peak_pfe > limit else "OK"
color = "red" if status == "BREACH" else "green"

print(f"Peak PFE:  €{peak_pfe:,.0f}")
print(f"Limit:     €{limit:,.0f}")
print(f"Status:    {status}")

## Next Steps Already Coded (Sprint 2 & 3)
- Full backtesting module (2018–2025)
- Limits & EWI engine
- Governance playbook

→ See `docs/playbook.md` for methodology & regulatory alignment